In [1]:
###### Import fundamentals
import numpy as np
import pandas as pd
import seaborn as sns
import re

# Import nltk and download punkt, wordnet
import nltk

# Import word_tokenize and stopwords from nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer 
from nltk.tag import pos_tag


# Sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.model_selection import cross_val_score


# I will keep the resulting plots
%matplotlib inline

# Enable Jupyter Notebook's intellisense
%config IPCompleter.greedy=True

# We want to see whole content (non-truncated)
pd.set_option('display.max_colwidth', -1)

In [2]:
# Load the tweets
tweets = pd.read_csv("Dataset.csv")

# Print the first five rows
display(tweets.head())

,Tweets,Label
0,@lynn93630469 Support my little sister in her school by buying a laptop for her Online Class and a stable wifi connection in our home. She badly needs it because she only uses an android phone and a data connection. 😊 Whenever she has something to encode she borrows laptop from our cousin's.,Neutral
1,"Yan, tama yan. Dapat lang na nasa #1 &amp; #2 ang tags natin, aba! Pambawi sa puyat ko, hahaha. Alas-kwatro na ako nakatulog kanina dahil sa SBEN19 MAPA tapos gumising ng 7 AM para sa online class 😬 STREAM SBEN19MAPA @SB19Official #SBEN19MAPAOutNow #SBNineteenAtKalayaan2021",Neutral
2,Kabi-kbila na ang utang ko dahil sa online class n to🥺🥺 panload pa lng di ko n afford🥺 Need some help🙏🙏🥺,Negative
3,Goodmorning🌞 Online class is real😂,Neutral
4,umay sa globe fiber. goodluck pag may online class na talaga.,Negative


In [3]:
#Open the text file containing the Filipino Stopwords based from https://github.com/stopwords-iso/stopwords-tl

file = open("StopWords/flstopwords.txt", "r", encoding="utf8")
flstopwords = file.read().split("\n")
file.close()

In [4]:
"""
Created on Wed Aug  5 15:39:20 2020

@author: bhasfe
"""
def process_tweets(tweet):
            
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"can't", "can not", tweet)
    tweet = re.sub(r"n't", " not", tweet)
    tweet = re.sub(r"'ve", " have", tweet)
    tweet = re.sub(r"'ll", " will", tweet)
    tweet = re.sub(r"'re", " are", tweet)
    
    tweet = re.sub(r"'di", "hindi", tweet)
    tweet = re.sub(r"di", "hindi", tweet)
    
    # Remove links
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    
    #remove numbers
    tweet = re.sub(r'\d','', tweet)
    
    # Remove mentions and hashtag
    tweet = re.sub(r'\@\w+|\#','', tweet)
   
    # clean the words
    clean = word_tokenize(tweet)

    # Remove the English stop words
    clean = [token for token in clean if token not in stopwords.words("english")]
    
    #Remove the Filipino stop words
    clean = [token for token in clean if token not in flstopwords]
    
    # Remove non-alphabetic characters and keep the words contains three or more letters
    clean = [token for token in clean if token.isalpha() and len(token)>2]
    
    clean = ' '.join(clean)
    return clean
    
# Call the function and store the result into a new column
tweets["Processed"] = tweets["Tweets"].str.lower().apply(process_tweets)

display(tweets[["Processed"]].head(15))

,Processed
0,support little sister school buying laptop online class stable wifi connection home badly needs uses android phone data connection whenever something encode borrows laptop cousin
1,yan tama yan lang nasa amp tags natin aba pambawi puyat hahaha nakatulog kanina sben mapa tapos gumising online class stream sbenmapa sbenmapaoutnow sbnineteenatkalayaan
2,utang online class panload lng need
3,online class
4,umay globe fiber goodluck pag online class talaga
5,mad last october first semester terpaksa jahindi online class ended class semester last thursday sin hindid deserve
6,online class ayoko mag enroll
7,sinusulit lang yung year online class law school palaging bahay lang
8,today using lot khursus online class watching stream movie lastly playing game wow day
9,know hinhindi tlaga pwede online class nakatulog lecture hahahahahahaha


In [85]:
def NormalizeWithPOS(text):
    # Lemmatization & Stemming according to POS tagging

    word_list = word_tokenize(text)
    rev = []
    lemmatizer = WordNetLemmatizer() 
    stemmer = PorterStemmer() 
    for word, tag in pos_tag(word_list):
        if tag.startswith('J'):
            w = lemmatizer.lemmatize(word, pos='a')
        elif tag.startswith('V'):
            w = lemmatizer.lemmatize(word, pos='v')
        elif tag.startswith('N'):
            w = lemmatizer.lemmatize(word, pos='n')
        elif tag.startswith('R'):
            w = lemmatizer.lemmatize(word, pos='r')
        else:
            w = word
        #w = stemmer.stem(w)
        rev.append(w)
    tweet = ' '.join(rev)
    return tweet

tweets["Processed"] = tweets["Processed"].apply(NormalizeWithPOS)
display(tweets[["Processed"]].head(15))

,Processed
0,support littl sister school buy laptop onlin class stabl wifi connect home badli need us android phone data connect whenev someth encod borrow laptop cousin
1,yan tama yan lang nasa amp tag natin aba pambawi puyat hahaha nakatulog kanina sben mapa tapo gumis onlin class stream sbenmapa sbenmapaoutnow sbnineteenatkalayaan
2,utang onlin class panload lng need
3,onlin class
4,umay globe fiber goodluck pag onlin class talaga
5,mad last octob first semest terpaksa jahindi onlin class end class semest last thursday sin hindid deserv
6,onlin class ayoko mag enrol
7,sinusulit lang yung year onlin class law school palag bahay lang
8,today use lot khursu onlin class watch stream movi lastli play game wow day
9,know hinhindi tlaga pwede onlin class nakatulog lectur hahahahahahaha


In [86]:
tfidf_stops = ["online", "class", "course", "learning", "learn","teach", "teaching", "distance", \
               "distancelearning", "education", "teacher", "student", "grade", "classes", "computer", "resource", \
               "onlineeducation", "onlinelearning", "school", "students", "class", "virtual", "eschool", "thing", \
               "virtuallearning", "educated", "educates", "teaches", "studies", "study", "semester", "elearning", \
               "teachers", "lecturer", "lecture", "amp", "academic", "admission", "academician", "account", "action",\
               "add", "app", "announcement", "application", "adult", "classroom", "system", "video", "essay", "training", \
               "homework","work","assignment", "paper", "get", "math", "project", "science", "physics", "lesson", "schools", \
               "courses", "assignments", "know", "instruction","email", "discussion","home", "college", "exam", "university", \
               "use", "fall", "term", "proposal", "one", "review", "proposal", "calculus", "search", "research", "algebra", \
               "internet", "remote", "remotelearning"]

enstopwords = set(stopwords.words('english'))
        
# Initialize a Tf-idf Vectorizer
vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, stop_words=enstopwords and flstopwords and tfidf_stops)

# Fit and transform the vectorizer corpus = [str (item) for item in corpus]
tfidf_matrix = vectorizer.fit_transform(str (item) for item in tweets["Processed"])

# Let's see what we have
tfidf_matrix

# Create a DataFrame for tf-idf vectors and display the first five rows
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns= vectorizer.get_feature_names())
display(tfidf_df.head())

,aaaaaaa,aabot,aabsent,aadjust,aalaga,aantay,aaral,aaralin,aasawa,aattend,...,ªðÿ,âœš,œðÿ,𝐂𝐡𝐚𝐢𝐧,𝐌𝐚𝐧𝐚𝐠𝐞𝐦𝐞𝐧𝐭,𝐎𝐧𝐥𝐢𝐧𝐞,𝐏𝐀𝐒𝐈𝐀,𝐏𝐮𝐛𝐥𝐢𝐜,𝐒𝐮𝐩𝐩𝐥𝐲,𝐓𝐫𝐚𝐢𝐧𝐢𝐧𝐠
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
# Select the features and the target
X = tfidf_matrix
y = tweets["Label"]

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = .20)

In [89]:
def confussionMatrix(cl,X_test,y_test):
    # Predict the labels
    y_pred = cl.predict(X_test)
    
    # Print the Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix\n")
    print(cm)
    
    # Print the Classification Report
    cr = classification_report(y_test, y_pred)
    print("\n\nClassification Report\n")
    print(cr)
    
    return

In [90]:
#1 Multinomial Naive Bayes
mnb = MultinomialNB()
#train our algorithm
mnb.fit(X_train, y_train)
#Test the trained classifier
predicted_class = mnb.predict(X_test)
#print('Accuracy of MNB for this dataset: %3.2f' %  (accuracy_score(y_test, predicted_class)*100))
confussionMatrix(mnb,X_test,y_test)

Confusion Matrix

[[679 246   0]
 [236 672   0]
 [ 47 121   0]]


Classification Report

              precision    recall  f1-score   support

    Negative       0.71      0.73      0.72       925
     Neutral       0.65      0.74      0.69       908
    Positive       0.00      0.00      0.00       168

    accuracy                           0.68      2001
   macro avg       0.45      0.49      0.47      2001
weighted avg       0.62      0.68      0.65      2001



C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [91]:
#2 Complement Naive Bayes
cnb = ComplementNB()

cnb.fit(X_train, y_train)
predicted_class = cnb.predict(X_test)
#print('Accuracy of MNB for this dataset: %3.2f' %  (accuracy_score(y_test, predicted_class)*100))
confussionMatrix(cnb,X_test,y_test)

Confusion Matrix

[[694 172  59]
 [275 515 118]
 [ 44  72  52]]


Classification Report

              precision    recall  f1-score   support

    Negative       0.69      0.75      0.72       925
     Neutral       0.68      0.57      0.62       908
    Positive       0.23      0.31      0.26       168

    accuracy                           0.63      2001
   macro avg       0.53      0.54      0.53      2001
weighted avg       0.64      0.63      0.63      2001



In [92]:
#3 Bernoulli Naive Bayes classifier
nb = BernoulliNB()

# Fit the model
nb.fit(X_train, y_train)

predicted_class = nb.predict(X_test)
#print('Accuracy of Gaussian Naive Bayes for this dataset: %3.2f' %  (accuracy_score(y_test, predicted_class)*100))
confussionMatrix(nb,X_test,y_test)

Confusion Matrix

[[630 266  29]
 [194 664  50]
 [ 40 112  16]]


Classification Report

              precision    recall  f1-score   support

    Negative       0.73      0.68      0.70       925
     Neutral       0.64      0.73      0.68       908
    Positive       0.17      0.10      0.12       168

    accuracy                           0.65      2001
   macro avg       0.51      0.50      0.50      2001
weighted avg       0.64      0.65      0.64      2001



In [84]:
vectorizer.get_feature_names()

['aaaaaaa',
 'aabot',
 'aabsent',
 'aadjust',
 'aalaga',
 'aantay',
 'aaral',
 'aaralin',
 'aasawa',
 'aattend',
 'aaway',
 'aayo',
 'aayusin',
 'aba',
 'abala',
 'abi',
 'abl',
 'abot',
 'absent',
 'absolut',
 'abt',
 'abusado',
 'abutan',
 'abutin',
 'acad',
 'academ',
 'academi',
 'academicbreaknow',
 'academiceasenow',
 'academicfreez',
 'academicfreezenow',
 'acc',
 'accept',
 'access',
 'accessori',
 'acct',
 'ace',
 'ach',
 'achiev',
 'acr',
 'act',
 'activ',
 'activit',
 'actual',
 'adapt',
 'address',
 'adel',
 'adjust',
 'administr',
 'admit',
 'adopt',
 'ador',
 'advanc',
 'advantag',
 'advic',
 'advis',
 'affect',
 'afford',
 'afraid',
 'afternoon',
 'aga',
 'agad',
 'agahan',
 'age',
 'agent',
 'ago',
 'agre',
 'ahaha',
 'ahahah',
 'ahahaha',
 'ahahahah',
 'ahahahaha',
 'ahahahahaha',
 'ahead',
 'ahh',
 'ahhaha',
 'ahhahahaha',
 'ahhh',
 'ahindi',
 'ahindib',
 'aigoo',
 'aim',
 'aint',
 'air',
 'aircon',
 'akala',
 'akalain',
 'akong',
 'akoo',
 'akooo',
 'akoooo',
 'akooo

In [13]:
#1 Multinomial Naive Bayes


In [14]:
#2 Complement Naive Bayes


In [27]:
#3 Bernoulli Naive Bayes


In [93]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resample, y_resample = ros.fit_resample(X_train, y_train)

In [ ]:
y_resample.value_counts()

In [94]:
#1 Multinomial Naive Bayes
mnb = MultinomialNB()

#train our algorithm
mnb.fit(X_resample, y_resample)

#Test the trained classifier
predicted_class = mnb.predict(X_test)
print('Accuracy of MNB for this dataset: %3.2f' %  (accuracy_score(y_test, predicted_class)*100))

Accuracy of MNB for this dataset: 61.17


In [95]:
cnb = ComplementNB()

cnb.fit(X_resample, y_resample)
predicted_class = cnb.predict(X_test)
print('Accuracy of MNB for this dataset: %3.2f' %  (accuracy_score(y_test, predicted_class)*100))

Accuracy of MNB for this dataset: 58.67


In [96]:
#3 Bernoulli Naive Bayes classifier
nb = BernoulliNB()

# Fit the model
nb.fit(X_resample, y_resample)

predicted_class = nb.predict(X_test)
print('Accuracy of Gaussian Naive Bayes for this dataset: %3.2f' %  (accuracy_score(y_test, predicted_class)*100))

Accuracy of Gaussian Naive Bayes for this dataset: 63.92


In [100]:
words = "ang saya ng online class"
clean = process_tweets(str.lower(words))
clean = NormalizeWithPOS(clean)
words = vectorizer.transform([clean])

In [101]:
mnb.predict(words)

array(['Positive'], dtype='<U8')

In [102]:
cnb.predict(words)

array(['Positive'], dtype='<U8')

In [103]:
nb.predict(words)

array(['Positive'], dtype='<U8')

In [97]:
#Multinomial Naive Bayes 
confussionMatrix(mnb,X_test,y_test)

Confusion Matrix

[[671 161  93]
 [250 467 191]
 [ 38  44  86]]


Classification Report

              precision    recall  f1-score   support

    Negative       0.70      0.73      0.71       925
     Neutral       0.69      0.51      0.59       908
    Positive       0.23      0.51      0.32       168

    accuracy                           0.61      2001
   macro avg       0.54      0.58      0.54      2001
weighted avg       0.66      0.61      0.62      2001



In [98]:
#Complement Naive Bayes 
confussionMatrix(cnb,X_test,y_test)

Confusion Matrix

[[668 154 103]
 [280 412 216]
 [ 36  38  94]]


Classification Report

              precision    recall  f1-score   support

    Negative       0.68      0.72      0.70       925
     Neutral       0.68      0.45      0.54       908
    Positive       0.23      0.56      0.32       168

    accuracy                           0.59      2001
   macro avg       0.53      0.58      0.52      2001
weighted avg       0.64      0.59      0.60      2001



In [99]:
#Bernoulli Naive Bayes
confussionMatrix(nb,X_test,y_test)

Confusion Matrix

[[633 223  69]
 [193 586 129]
 [ 37  71  60]]


Classification Report

              precision    recall  f1-score   support

    Negative       0.73      0.68      0.71       925
     Neutral       0.67      0.65      0.66       908
    Positive       0.23      0.36      0.28       168

    accuracy                           0.64      2001
   macro avg       0.54      0.56      0.55      2001
weighted avg       0.66      0.64      0.65      2001



In [104]:
def confusionmt(cl, X_test, y_test):
    # Predict the labels
    y_pred = cl.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)

    total = cm.sum().sum()
    accuracy = np.diag(cm).sum() / total
    accuracy = accuracy * 100
    accuracy = round(accuracy, 2)

    return accuracy

In [105]:
mnbacc = confusionmt(mnb, X_test, y_test)
cnbacc = confusionmt(cnb, X_test, y_test)
bnbacc = confusionmt(nb, X_test, y_test)

In [106]:
import pickle
accuracy = [mnbacc, cnbacc, bnbacc]
print(accuracy)

[61.17, 58.67, 63.92]


In [107]:
pickle.dump(accuracy, open("acc.pkl", 'wb'))
pickle.dump(mnb, open("MNB_model.pkl", 'wb'))
pickle.dump(cnb, open("CNB_model.pkl", 'wb'))
pickle.dump(nb, open("BNB_model.pkl", 'wb'))
pickle.dump(vectorizer, open("vectorizer.pkl", 'wb'))